In [2]:
from tensorflow.python.client import device_lib
import sys
import tensorflow as tf
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17772987797250503036
]


In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


[]

In [ ]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from IPython.display import display
from keras.preprocessing.image import img_to_array
from keras.callbacks import ModelCheckpoint

# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 200, 200
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 128
# Количество изображений для обучения
#nb_train_samples = 17500
nb_train_samples = 1000
# Количество изображений для проверки
nb_validation_samples = 300
# Количество изображений для тестирования
nb_test_samples = 0


# Количество классов изображений
nb_classes = 3
# Количество эпох для обучения
nb_epoch = 1000
# Количество каналов в изображении: RGB
img_channels = 3
# Названия классов из набора данных CIFAR-10
#classes=['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']

# Создаем последовательную модель
model = Sequential()
# Первый сверточный слой
model.add(Conv2D(32, (3, 3), padding='same',
                        input_shape=(200, 200, 3), activation='selu'))
# Первый слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Слой регуляризации Dropout
model.add(Dropout(0.2))

# Третий сверточный слой
model.add(Conv2D(32, (3, 3), padding='same', activation='selu'))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Слой регуляризации Dropout
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same', activation='selu'))
# Четвертый сверточный слой
model.add(Conv2D(64, (3, 3), activation='selu'))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Слой регуляризации Dropout
model.add(Dropout(0.25))


# Слой преобразования данных из 2D представления в плоское
model.add(Flatten())
# Полносвязный слой для классификации
model.add(Dense(1024, activation='selu'))
# Слой регуляризации Dropout
model.add(Dropout(0.5))
# Выходной полносвязный слой
model.add(Dense(nb_classes, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

print(model.summary())

aug_gen = ImageDataGenerator(rescale=1./255,
                             rotation_range=15,
                             width_shift_range=0.15,
                             height_shift_range=0.15,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')


datagen = ImageDataGenerator(rescale=1./255)
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Генератор данных для обучения на основе изображений из каталога
train_generator = aug_gen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Генератор данных для проверки на основе изображений из каталога
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Генератор данных для тестирования на основе изображений из каталога
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Обучаем модель с использованием генераторов
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=nb_epoch,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save('first_try.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 32)      9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 64)        18496     
__________

Epoch 46/1000
7/7 [==============================] - 44s 6s/step - loss: 1.0984 - acc: 0.3304 - val_loss: 1.0984 - val_acc: 0.3398
Epoch 47/1000
7/7 [==============================] - 44s 6s/step - loss: 1.0989 - acc: 0.3158 - val_loss: 1.0988 - val_acc: 0.3320
Epoch 48/1000
7/7 [==============================] - 42s 6s/step - loss: 1.0981 - acc: 0.3292 - val_loss: 1.0983 - val_acc: 0.3106
Epoch 49/1000
7/7 [==============================] - 44s 6s/step - loss: 1.0944 - acc: 0.3571 - val_loss: 1.0906 - val_acc: 0.3516
Epoch 50/1000
7/7 [==============================] - 44s 6s/step - loss: 1.0969 - acc: 0.3527 - val_loss: 1.0975 - val_acc: 0.4102
Epoch 51/1000
7/7 [==============================] - 44s 6s/step - loss: 1.0980 - acc: 0.3292 - val_loss: 1.0970 - val_acc: 0.3945
Epoch 52/1000
7/7 [==============================] - 41s 6s/step - loss: 1.0985 - acc: 0.3236 - val_loss: 1.0990 - val_acc: 0.3398
Epoch 53/1000
7/7 [==============================] - 42s 6s/step - loss: 1.0973 - a

7/7 [==============================] - 42s 6s/step - loss: 0.7830 - acc: 0.6719 - val_loss: 0.7854 - val_acc: 0.6515
Epoch 109/1000
7/7 [==============================] - 44s 6s/step - loss: 0.7519 - acc: 0.6652 - val_loss: 0.7848 - val_acc: 0.6445
Epoch 110/1000
7/7 [==============================] - 41s 6s/step - loss: 0.7918 - acc: 0.6290 - val_loss: 0.7596 - val_acc: 0.6797
Epoch 111/1000
7/7 [==============================] - 44s 6s/step - loss: 0.8710 - acc: 0.6016 - val_loss: 0.7863 - val_acc: 0.6602
Epoch 112/1000
7/7 [==============================] - 44s 6s/step - loss: 0.8481 - acc: 0.5971 - val_loss: 0.7982 - val_acc: 0.6836
Epoch 113/1000
7/7 [==============================] - 42s 6s/step - loss: 0.7732 - acc: 0.6763 - val_loss: 0.8200 - val_acc: 0.6364
Epoch 114/1000
7/7 [==============================] - 41s 6s/step - loss: 0.8010 - acc: 0.6543 - val_loss: 0.8860 - val_acc: 0.5742
Epoch 115/1000
7/7 [==============================] - 44s 6s/step - loss: 0.7836 - acc: 0.6

7/7 [==============================] - 44s 6s/step - loss: 0.5848 - acc: 0.7411 - val_loss: 1.0234 - val_acc: 0.5820
Epoch 171/1000
7/7 [==============================] - 44s 6s/step - loss: 0.6485 - acc: 0.7277 - val_loss: 0.7183 - val_acc: 0.6641
Epoch 172/1000
7/7 [==============================] - 41s 6s/step - loss: 0.5821 - acc: 0.7410 - val_loss: 0.8345 - val_acc: 0.6562
Epoch 173/1000
7/7 [==============================] - 42s 6s/step - loss: 0.5502 - acc: 0.7679 - val_loss: 0.7312 - val_acc: 0.7045
Epoch 174/1000
7/7 [==============================] - 44s 6s/step - loss: 0.5338 - acc: 0.7667 - val_loss: 0.9372 - val_acc: 0.6211
Epoch 175/1000
7/7 [==============================] - 41s 6s/step - loss: 0.5882 - acc: 0.7412 - val_loss: 0.7854 - val_acc: 0.6758
Epoch 176/1000
7/7 [==============================] - 44s 6s/step - loss: 0.5644 - acc: 0.7511 - val_loss: 0.6352 - val_acc: 0.7773
Epoch 177/1000
7/7 [==============================] - 45s 6s/step - loss: 0.5754 - acc: 0.7

7/7 [==============================] - 44s 6s/step - loss: 0.4161 - acc: 0.8304 - val_loss: 0.6064 - val_acc: 0.7852
Epoch 233/1000
7/7 [==============================] - 42s 6s/step - loss: 0.4200 - acc: 0.8158 - val_loss: 0.5002 - val_acc: 0.8182
Epoch 234/1000
7/7 [==============================] - 41s 6s/step - loss: 0.4562 - acc: 0.8088 - val_loss: 1.0861 - val_acc: 0.6836
Epoch 235/1000
7/7 [==============================] - 44s 6s/step - loss: 0.4738 - acc: 0.8114 - val_loss: 0.6461 - val_acc: 0.7422
Epoch 236/1000
7/7 [==============================] - 44s 6s/step - loss: 0.4117 - acc: 0.8404 - val_loss: 0.4700 - val_acc: 0.8086
Epoch 237/1000
7/7 [==============================] - 41s 6s/step - loss: 0.4461 - acc: 0.8226 - val_loss: 0.5960 - val_acc: 0.7891
Epoch 238/1000
7/7 [==============================] - 42s 6s/step - loss: 0.4080 - acc: 0.8292 - val_loss: 0.4130 - val_acc: 0.8409
Epoch 239/1000
7/7 [==============================] - 44s 6s/step - loss: 0.4008 - acc: 0.8